In [1]:
from gekko import GEKKO

g = GEKKO(remote=False)

i_max = 3
j_max = 4
k_max = 3
alfa_max = 5

GHG1_values = [1, 2, 3]
GHG1 = [g.Param(value=GHG1_values[i]) for i in range(i_max)] 

GHG2_values = [[1, 2, 3, 10],
               [4, 5, 6, 10],
               [7, 8, 9, 10]]
GHG2 = [[g.Param(value=GHG2_values[i][j]) for j in range(j_max)] for i in range(i_max)]

s_values = [1, 2, 3]
s = [g.Param(value=s_values[i]) for i in range(i_max)]

SI_values = [1, 2, 3]
SI = [g.Param(value=SI_values[i]) for i in range(i_max)]

f_values = [1, 2, 3, 10]
f = [g.Param(value=f_values[j]) for j in range(j_max)]

c_values = [1, 2, 3, 10, 100]
c = [g.Param(value=c_values[alfa]) for alfa in range(alfa_max)]

d_values = [1, 2, 3, 10]
d = [g.Param(value=d_values[j]) for j in range(j_max)]

r_values = [[1, 2, 3, 10],
            [4, 5, 6, 10],
            [7, 8, 9, 10]]
r = [[g.Param(value=r_values[k][j]) for j in range(j_max)] for k in range(k_max)]

L = [g.Var(value=1, lb=0, ub=10) for j in range(j_max)]
Y = [[g.Var(value=1, lb=0, ub=10) for j in range(j_max)] for i in range(i_max)]
U = [g.Var(value=1, lb=0, ub=10) for j in range(j_max)]

for k in range(k_max):
    for j in range(j_max):
        g.Equation(U[j] <= sum(Y[i][j] / r[k][j] for i in range(i_max)))

for i in range(i_max):
    g.Equation(sum(Y[i][j] for j in range(j_max)) <= U[i] + s[i])

for j in range(j_max):
    g.Equation(U[j] == 0)

for j in range(j_max):
    g.Equation(L[j] + sum(U[k] for k in range(k_max)) >= d[j])

for alfa in range(alfa_max):
    g.Equation(sum(U[k] for k in range(k_max)) <= c[alfa])

objective1 = sum(f[j] * L[j] for j in range(j_max))
objective2 = sum(GHG1[i] * U[i] + sum(GHG2[i][j] * Y[i][j] for j in range(j_max)) for i in range(i_max))
objective3 = sum(SI[i] * U[i] for i in range(i_max))

min_objective = g.Var()
max_objective = g.Var()

g.Equation(min_objective >= objective1)
g.Equation(min_objective >= objective2)
g.Equation(max_objective == -objective3)

g.Minimize(min_objective + max_objective)

g.solve()

print('L:', [(j+1, L[j].value[0]) for j in range(j_max)])
print()
print('Y:', [[(i+1, j+1, Y[i][j].value[0]) for j in range(j_max)] for i in range(i_max)])
print()
print('U:', [(j+1, U[j].value[0]) for j in range(j_max)])
print()
print('Objective:', g.options.OBJFCNVAL)


 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  94
   Intermediates:  0
   Connections  :  0
   Equations    :  32
   Residuals    :  32
 
 Number of state variables:    48
 Number of total equations: -  31
 Number of slack variables: -  26
 ---------------------------------------
 Degrees of freedom       :    -9
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse